In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/spamham/spam.csv",encoding='Windows-1252')
df.info()


In [ ]:

df = df[['v1','v2']]
df.head()

nameDict = {"v1":"label","v2":"message"}
nameDict
df = df.rename(columns=nameDict)
df


In [ ]:
X=list(df['message'])



In [ ]:
y=list(df['label'])



In [ ]:
dictOne = {"spam":1,"ham":0}
y = list(df['label'].replace(dictOne))


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# !pip install transformers


In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer


In [ ]:
train_encodings = tokenizer(X_train,truncation = True,padding=True)
test_encodings = tokenizer(X_test,truncation=True,padding=True)
train_encodings = dict(train_encodings)
test_encodings = dict(test_encodings)


In [ ]:
import tensorflow as tf


In [ ]:

train_dataset = tf.data.Dataset.from_tensor_slices((train_encodings,y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((test_encodings,y_test))

train_dataset



In [ ]:
#!pip install transformers==4.18.0


In [ ]:

from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification


In [ ]:
from transformers.trainer_tf import TFTrainer


In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()



In [ ]:

trainer.evaluate(test_dataset)

